# Uno multiplayer

#### Import all library

In [1]:
import pygame
import socket
import time
import asyncio
import threading

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


#### Button class

Taken at :
https://github.com/russs123/pygame_tutorials/blob/main/Button/button.py

By russs123 : 
https://github.com/russs123

In [2]:
class Button():
    def __init__(self, x, y, image, scale):
        width = image.get_width()
        height = image.get_height()
        self.image = pygame.transform.scale(image, (int(width * scale), int(height * scale)))
        self.rect = self.image.get_rect()
        self.rect.topleft = (x, y)
        self.clicked = False

    def draw(self, surface):
        action = False
        #get mouse position
        pos = pygame.mouse.get_pos()

        #check mouseover and clicked conditions
        if self.rect.collidepoint(pos):
            if pygame.mouse.get_pressed()[0] == 1 and self.clicked == False:
                self.clicked = True
                action = True

        if pygame.mouse.get_pressed()[0] == 0:
            self.clicked = False

        #draw button on screen
        surface.blit(self.image, (self.rect.x, self.rect.y))

        return action

#### Pygame initialisation

In [3]:
pygame.init()
screen = pygame.display.set_mode((1000,500)) #,pygame.FULLSCREEN
clock = pygame.time.Clock()
run = True
baseFont = pygame.font.Font(None,32)
pygame.display.set_caption('Uno multiplayer')

#### Pygame quit event

In [4]:
def quitEvent():
    run = False
    send(DISCONNECT_MESSAGE)
    pygame.quit()

#### Socket and var initialisation

In [5]:
HEADER = 64
PORT = 8080
SERVER = "localhost"
FORMAT = 'utf-8'
DISCONNECT_MESSAGE = "!DISCONNECT"
ADDR = (SERVER, PORT)
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(ADDR)
connected = True
currentMessage = ["none","none","none"]
currentRoom = ""
username = ""
players = []

#### Socket loop

In [6]:
def start_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

#### Socket get message

In [7]:
async def socket():
    global currentMessage
    global run
    while run:
        message = client.recv(512).decode(FORMAT)
        currentMessage = message.split(',')

#### Send function

In [8]:
def send(msg):
    message = msg.encode(FORMAT)
    msg_lenght = len(message)
    send_lenght = str(msg_lenght).encode(FORMAT)
    send_lenght += b' ' * (HEADER - len(send_lenght))
    client.send(send_lenght)
    client.send(message)

#### Load assets

In [9]:
newGameBtn = pygame.image.load('./assets/Buttons/Red/New Game 192x48.png').convert_alpha()
optionBtn = pygame.image.load('./assets/Buttons/Red/Options 192x48.png').convert_alpha()
quitBtn = pygame.image.load('./assets/Buttons/Red/Quit 192x48.png').convert_alpha()
exitBtn = pygame.image.load('./assets/Buttons/Red/Exit 192x48.png').convert_alpha()
audioBtn = pygame.image.load('./assets/Buttons/Red/Audio 192x48.png').convert_alpha()
newBtn = pygame.image.load('./assets/Buttons/Red/New 192x48.png').convert_alpha()
joinBtn = pygame.image.load('./assets/Buttons/Red/Load Game 192x48.png').convert_alpha()
quitGameBtn = pygame.image.load('./assets/Buttons/Red/Quit Game 192x48.png').convert_alpha()
startGameBtn = pygame.image.load('./assets/Buttons/Red/Start Game 192x48.png').convert_alpha()

#### Text Display

In [10]:
textCol = (255,255,255)
colorTextRect = (200,200,200)
def drawText(text, font, textCol, x, y):
    img = font.render(text, True, textCol)
    screen.blit(img, (x, y))

#### Option menu

In [11]:
def optionMenu():
    localRun = True
    abilityToClick = False
    x, y = pygame.display.get_surface().get_size()
    audioSettings = Button(x/2-((audioBtn.get_width()/2)*1.5), y/3-((audioBtn.get_height()/2)*1.5), audioBtn, 1.5)
    exitMenu = Button(x/2-((exitBtn.get_width()/2)*1.5), 2*y/3-((exitBtn.get_height()/2)*1.5), exitBtn, 1.5)
    
    while localRun:
        pygame.display.flip()
        screen.fill((0, 0, 0))
        events = pygame.event.get()
        if audioSettings.draw(screen) and abilityToClick:
            abilityToClick = False
            print('Audio')
        if exitMenu.draw(screen) and abilityToClick:
            abilityToClick = False
            localRun = False 
        abilityToClick = True
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quitEvent()

#### Join game menu

In [12]:
def joinMenu():
    
    global currentMessage

    global currentRoom

    global username

    errorMessage = ""
    
    localRun = True

    abilityToClick = False

    screen.fill((0, 0, 0))

    x, y = pygame.display.get_surface().get_size()
    
    #create rectangle as text box
    codeTextBox = pygame.Rect(x/2-((joinBtn.get_width()/2)*1.5),3*y/6-((newBtn.get_height()/2)*1.5)+ (newBtn.get_height()/2)-7, 105,joinBtn.get_height())
    usernameTextBox = pygame.Rect(x/2-((joinBtn.get_width()/2)*1.5),y/6-((newBtn.get_height()/2)*1.5)+ (newBtn.get_height()/2)-7, joinBtn.get_width()*1.5,joinBtn.get_height())
    #Variable for the text box (color of the rectangle and state of the box selected or not)
    activeCode = False
    activeUsername = False
    
    #String for the text box
    code = ""
    
    joinRoom = Button(x/2-((joinBtn.get_width()/2)*1.5), 4*y/6-((joinBtn.get_height()/2)*1.5), joinBtn, 1.5)
    newRoom = Button(x/2-((newBtn.get_width()/2)*1.5), 2*y/6-((newBtn.get_height()/2)*1.5), newBtn, 1.5)
    exitMenu = Button(x/2-((exitBtn.get_width()/2)*1.5), 5*y/6-((exitBtn.get_height()/2)*1.5), exitBtn, 1.5)
        
    
    while localRun:
        UI_REFRESH_RATE = clock.tick(60)/1000
        screen.fill((0, 0, 0))
        #draw the text box
        pygame.draw.rect(screen,colorTextRect,codeTextBox,2)
        pygame.draw.rect(screen,colorTextRect,usernameTextBox,2)
        #write text
        label = baseFont.render(errorMessage, 1, textCol)
        screen.blit(label, (x/2-((newBtn.get_width()/2)*1.5), y/9-((newBtn.get_height()/2)*1.5) + newBtn.get_height()/2))
        label = baseFont.render("Your username :", 1, textCol)
        screen.blit(label, (x/2-((newBtn.get_width()/2)*1.5) - 250, y/6-((newBtn.get_height()/2)*1.5) + newBtn.get_height()/2))
        label = baseFont.render("Room code :", 1, textCol)
        screen.blit(label, (x/2-((joinBtn.get_width()/2)*1.5) - 250, 3*y/6-((joinBtn.get_height()/2)*1.5) + joinBtn.get_height()/2))
        #set the location or where we can write
        drawText(username, baseFont, textCol, x/2-(newBtn.get_width()/2)*1.5 ,y/6-((newBtn.get_height()/2)*1.5)+ newBtn.get_height()/2)
        drawText(code, baseFont, textCol, x/2-(newBtn.get_width()/2)*1.5 ,3*y/6-((newBtn.get_height()/2)*1.5)+ newBtn.get_height()/2)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quitEvent()
            #check if a text box is selected
            if event.type == pygame.MOUSEBUTTONDOWN:
                if codeTextBox.collidepoint(event.pos):
                    activeCode = True
                else:
                    activeCode = False
                if usernameTextBox.collidepoint(event.pos): 
                    activeUsername = True
                else:
                    activeUsername = False
            #write in the code text box
            if activeCode == True:     
                if event.type == pygame.TEXTINPUT:
                    if len(code) < 5:
                        if event.text != "," and event.text != ".":
                            code += event.text.upper()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_BACKSPACE:
                        code = code[:-1]
            #write in the username text box            
            if activeUsername == True:     
                if event.type == pygame.TEXTINPUT:
                    if len(username) < 13:
                        if event.text != "," and event.text != ".":
                            username += event.text
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_BACKSPACE:
                        username = username[:-1]
        if newRoom.draw(screen) and abilityToClick:
            abilityToClick = False
            if len(username) > 0:
                send(f"{username},createRoom,-1")
                while currentMessage[0] != "error" or currentMessage[0] != "joinNow":
                    if currentMessage[0] == "error":
                        currentRoom = currentMessage[1]
                        errorMessage = "Cannot create a room."
                        break
                    elif currentMessage[0] == "joinNow":
                        currentRoom = currentMessage[1]
                        errorMessage = ""
                        lobbyMenu()
                        break
                currentMessage = ["none","none","none"]
            else:
                errorMessage = "Please enter a username."
        if joinRoom.draw(screen) and abilityToClick:
            abilityToClick = False
            if len(username) > 0 and len(code) > 0:
                send(f"{username},joinRoom,{code}")
                while currentMessage[0] != "error" or currentMessage[0] != "joinNow":
                    if currentMessage[0] == "error":
                        currentRoom = currentMessage[1]
                        errorMessage = "Cannot join this room (may be full or not exist)."
                        break
                    elif currentMessage[0] == "joinNow":
                        currentRoom = currentMessage[1]
                        errorMessage = ""
                        lobbyMenu()
                        break
                currentMessage = ["none","none","none"]
            else:
                errorMessage = "Please enter a username and a room code."
        if exitMenu.draw(screen) and abilityToClick:
            abilityToClick = False
            localRun = False
        abilityToClick = True
        pygame.display.flip()
        pygame.display.update()
        

#### Current lobby menu

In [13]:
def lobbyMenu():
    localRun = True

    global players

    global currentRoom

    global currentMessage

    global username
    
    abilityToClick = False

    x, y = pygame.display.get_surface().get_size()
    startGame = Button(x/2-((startGameBtn.get_width()/2)*1.5), y/3-((startGameBtn.get_height()/2)*1.5), startGameBtn, 1.5)
    quitGame = Button(x/2-((quitGameBtn.get_width()/2)*1.5), 2*y/3-((quitGameBtn.get_height()/2)*1.5), quitGameBtn, 1.5)
    
    send(f"{username},getPlayers,{currentRoom}")
    while localRun:
        screen.fill((0, 0, 0))
        if currentMessage[0] == "setPlayers":
            players = []
            counter = 0
            for msg in currentMessage:
                if counter > 0:
                    players.append(msg)
                counter = counter + 1
            currentMessage = ["none","none","none"]
        label = baseFont.render(f"Current room : {currentRoom}", 1, textCol)
        screen.blit(label, (50, 50))
        counter = 0
        for p in players:
            counter = counter + 1
            label = baseFont.render(f"Player {counter} : {p}", 1, textCol)
            screen.blit(label, (50, 50*(counter+1)))
        if startGame.draw(screen) and abilityToClick:
            abilityToClick = False
            joinMenu()
        if quitGame.draw(screen) and abilityToClick:
            abilityToClick = False
            players = []
            send(f"{username},quitRoom,{currentRoom}")
            localRun = False            
        abilityToClick = True
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quitEvent()
        pygame.display.update()

#### Game table

In [ ]:
def gameTable():
    print("table")

#### Pygame main menu

In [14]:
def mainMenu():
    global run
    
    abilityToClick = False

    x, y = pygame.display.get_surface().get_size()
    newGame = Button(x/2-((newGameBtn.get_width()/2)*1.5), y/4-((newGameBtn.get_height()/2)*1.5), newGameBtn, 1.5)
    option = Button(x/2-((optionBtn.get_width()/2)*1.5), y/2-((optionBtn.get_height()/2)*1.5), optionBtn, 1.5)
    quit = Button(x/2-((quitBtn.get_width()/2)*1.5), 3*y/4-((quitBtn.get_height()/2)*1.5), quitBtn, 1.5)

    while run:
        screen.fill((0, 0, 0))
        if newGame.draw(screen) and abilityToClick:
            abilityToClick = False
            joinMenu()
        if option.draw(screen) and abilityToClick:
            abilityToClick = False
            optionMenu()
        if quit.draw(screen) and abilityToClick:
            abilityToClick = False
            run = False
        abilityToClick = True
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
        pygame.display.update()
    

#### Start listening on socket

In [15]:
new_loop = asyncio.new_event_loop()
t = threading.Thread(target=start_loop, args=(new_loop,))
t.start()

asyncio.run_coroutine_threadsafe(socket(), new_loop)

<Future at 0x2331db71f70 state=pending>

#### Pygame main loop

In [16]:
while run:

    screen.fill((0, 0, 0))

    mainMenu()

    pygame.display.flip()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False

    pygame.display.update()

quitEvent()

error: video system not initialized